In [ ]:
from torchvision.models import resnet18

/home/HubensN/miniconda3/envs/prune/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [ ]:
import torch
import torch.quantization

class PyTorchQuantizer:
    def __init__(self, model, quantization_strategy='dynamic', dtype=torch.qint8):
        self.model = model
        self.quantization_strategy = quantization_strategy
        self.dtype = dtype
        self.quantized_model = None

    def fuse_model(self):
        # Model-specific fusion. Replace 'conv', 'bn' with your model's layer names.
        # For example, if your model has a sequential block with conv-bn-relu:
        # torch.quantization.fuse_modules(self.model, ['conv', 'bn', 'relu'], inplace=True)
        pass

    def quantize(self):
        if self.quantization_strategy == 'dynamic':
            self.quantized_model = torch.quantization.quantize_dynamic(
                self.model, {torch.nn.Linear, torch.nn.LSTM, torch.nn.Conv2d}, dtype=self.dtype
            )
        elif self.quantization_strategy == 'static':
            self.model.eval()
            self.fuse_model()
            self.model.qconfig = torch.quantization.get_default_qconfig('fbgemm')
            torch.quantization.prepare(self.model, inplace=True)
            # Calibrate the model here with representative data
            torch.quantization.convert(self.model, inplace=True)
            self.quantized_model = self.model
        else:
            raise ValueError("Unsupported quantization strategy")

        return self.quantized_model

    def dequantize(self):
        if self.quantized_model is not None:
            return self.quantized_model.dequantize()
        else:
            raise ValueError("Model has not been quantized")

In [ ]:
# Example Usage
model = resnet18()  # Replace with your model
quantizer = PyTorchQuantizer(model, quantization_strategy='static')
quantized_model = quantizer.quantize()

# For static quantization, you would need to calibrate the model with some


/home/HubensN/miniconda3/envs/prune/lib/python3.9/site-packages/torch/ao/quantization/observer.py:214: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(
/home/HubensN/miniconda3/envs/prune/lib/python3.9/site-packages/torch/ao/quantization/observer.py:1209: UserWarning: must run observer before calling calculate_qparams.                                    Returning default scale and zero point 
  warnings.warn(


In [ ]:
import torch
import torch.quantization

class PyTorchQuantizer:
    def __init__(self, model, quantization_strategy='static', dtype=torch.qint8):
        self.model = model
        self.quantization_strategy = quantization_strategy
        self.dtype = dtype
        self.quantized_model = None

    def prepare_for_qat(self):
        if self.quantization_strategy == 'static':
            self.model.train()
            self.fuse_model()
            self.model.qconfig = torch.quantization.get_default_qat_qconfig('fbgemm')
            torch.quantization.prepare_qat(self.model, inplace=True)
        elif self.quantization_strategy == 'dynamic':
            # Dynamic quantization does not require special preparation for QAT
            pass
        else:
            raise ValueError("Unsupported quantization strategy")

    def fuse_model(self):
        # Model-specific fusion. This method should be tailored to your specific model architecture.
        # For example, for a typical CNN with conv-bn-relu:
        # torch.quantization.fuse_modules(self.model, ['conv', 'bn', 'relu'], inplace=True)
        pass

    def quantize_dynamic(self):
        if self.quantization_strategy == 'dynamic':
            self.quantized_model = torch.quantization.quantize_dynamic(
                self.model, {torch.nn.Linear, torch.nn.LSTM, torch.nn.Conv2d}, dtype=self.dtype
            )
        else:
            # For static quantization, the model is already prepared for QAT
            pass

    def convert_to_quantized(self):
        if self.quantization_strategy == 'static':
            torch.quantization.convert(self.model, inplace=True)
            self.quantized_model = self.model
        elif self.quantization_strategy == 'dynamic':
            # Dynamic quantization is applied on-the-fly, no further conversion is needed
            pass
        else:
            raise ValueError("Unsupported quantization strategy")

    def dequantize(self):
        if self.quantized_model is not None:
            return self.quantized_model.dequantize()
        else:
            raise ValueError("Model has not been quantized")

# Example Usage
model = resnet18()  # Replace with your model
model.eval();
model = torch.quantization.fuse_modules(model, [['conv1', 'bn1', 'relu']], inplace=True)

quantizer = PyTorchQuantizer(model, quantization_strategy='static')
quantizer.prepare_for_qat()
# Train the model here with QuantizationAwareTrainingCallback
quantizer.convert_to_quantized()


/home/HubensN/miniconda3/envs/prune/lib/python3.9/site-packages/torch/ao/quantization/observer.py:214: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(
/home/HubensN/miniconda3/envs/prune/lib/python3.9/site-packages/torch/ao/quantization/utils.py:310: UserWarning: must run observer before calling calculate_qparams. Returning default values.
  warnings.warn(


In [ ]:
#| include: false
from fastai.vision.all import *
from fastai.callback.all import *
from fasterai.sparse.all import *

import torch
import torch.nn as nn
import torch.nn.functional as F

path = untar_data(URLs.PETS)
files = get_image_files(path/"images")

def label_func(f): return f[0].isupper()

dls = ImageDataLoaders.from_name_func(path, files, label_func, item_tfms=Resize(64))

In [ ]:
model = resnet18()

In [ ]:
x, y = dls.one_batch()

In [ ]:
learn = Learner(dls, model)

In [ ]:
learn.fit(1)

epoch,train_loss,valid_loss,time
0,0.659714,0.540446,00:09


In [ ]:
learn.model.to('cpu')

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
model = model.half()

In [ ]:
%%timeit
learn.model(x.to('cpu'))

RuntimeError: Input type (torch.FloatTensor) and weight type (torch.HalfTensor) should be the same or input should be a MKLDNN tensor and weight is a dense tensor

In [ ]:
model = model.to('cpu')

In [ ]:
x = x.to('cpu')

In [ ]:
torch.__version__

'2.0.1+cu117'

In [ ]:
import torchvision.models as models
import torch.quantization

model = models.resnet18(pretrained=True)
model.eval()
model.to('cpu')

# Fuse Conv, BN, ReLU
model = torch.quantization.fuse_modules(model, [['conv1', 'bn1', 'relu']], inplace=True)

# Specify quantization configuration
model.qconfig = torch.quantization.default_qconfig

# Prepare and convert model to quantized version
torch.quantization.prepare(model, inplace=True)
# Calibrate with representative data here
torch.quantization.convert(model, inplace=True)

# Test run
with torch.no_grad():
    sample_input = torch.rand(1, 3, 224, 224, dtype=torch.int8)
    output = model(sample_input.cpu())

RuntimeError: "check_uniform_bounds" not implemented for 'Char'

In [ ]:
torch.quantization.default_qconfig

QConfig(activation=functools.partial(<class 'torch.ao.quantization.observer.MinMaxObserver'>, quant_min=0, quant_max=127){}, weight=functools.partial(<class 'torch.ao.quantization.observer.MinMaxObserver'>, dtype=torch.qint8, qscheme=torch.per_tensor_symmetric){})

In [ ]:
from fasterai.misc.all import *

In [ ]:
import torch
import torchvision

class QuantizedModel(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model_fp32 = model
        self.quant = torch.quantization.QuantStub()
        self.dequant = torch.quantization.DeQuantStub()
        
    def forward(self, x):
        x = self.quant(x)
        x = self.model_fp32(x)
        x = self.dequant(x)
        return x
        
def fuse_resnet18(model):
    torch.quantization.fuse_modules(model, [["conv1", "bn1", "relu"]], inplace=True)
    for module_name, module in model.named_children():
        if "layer" in module_name:
            for basic_block_name, basic_block in module.named_children():
                torch.quantization.fuse_modules(basic_block, [["conv1", "bn1", "relu"], ["conv2", "bn2"]], inplace=True)
                for sub_block_name, sub_block in basic_block.named_children():
                    if sub_block_name == "downsample":
                        torch.quantization.fuse_modules(sub_block, [["0", "1"]], inplace=True)


        
def fuse_modules_iteratively(model):
    # Define common fusion patterns
    fusion_patterns = [["conv", "bn", "relu"], ["conv", "bn"], ["conv1", "bn1", "relu"], ["conv2", "bn2"], ["0", "1"]]

    # Create a queue to hold model modules for processing
    modules_queue = [model]

    while modules_queue:
        module = modules_queue.pop(0)

        # Fuse applicable modules at the current level
        for pattern in fusion_patterns:
            if all([hasattr(module, name) for name in pattern]):
                torch.quantization.fuse_modules(module, [pattern], inplace=True)

        # Add child modules to the queue
        modules_queue.extend(list(module.children()))


model = torchvision.models.resnet18()
model.eval();

#bn = BN_Folder()
#model = bn.fold(model)

#print(model)
fuse_resnet18(model)
quantized_model = QuantizedModel(model)

backend = "fbgemm"
model.qconfig = torch.quantization.get_default_qconfig(backend)
torch.backends.quantized.engine = backend
model_static_quantized = torch.quantization.prepare(quantized_model, inplace=False)
model_static_quantized = torch.quantization.convert(model_static_quantized, inplace=False)

inputs = torch.ones((1,3,224,224))

model_static_quantized(inputs)

/home/HubensN/miniconda3/envs/prune/lib/python3.9/site-packages/torch/ao/quantization/observer.py:1209: UserWarning: must run observer before calling calculate_qparams.                                    Returning default scale and zero point 
  warnings.warn(


NotImplementedError: Could not run 'quantized::conv2d_relu.new' with arguments from the 'CPU' backend. This could be because the operator doesn't exist for this backend, or was omitted during the selective/custom build process (if using custom build). If you are a Facebook employee using PyTorch on mobile, please visit https://fburl.com/ptmfixes for possible resolutions. 'quantized::conv2d_relu.new' is only available for these backends: [QuantizedCPU, QuantizedCUDA, BackendSelect, Python, FuncTorchDynamicLayerBackMode, Functionalize, Named, Conjugate, Negative, ZeroTensor, ADInplaceOrView, AutogradOther, AutogradCPU, AutogradCUDA, AutogradXLA, AutogradMPS, AutogradXPU, AutogradHPU, AutogradLazy, AutogradMeta, Tracer, AutocastCPU, AutocastCUDA, FuncTorchBatched, FuncTorchVmapMode, Batched, VmapMode, FuncTorchGradWrapper, PythonTLSSnapshot, FuncTorchDynamicLayerFrontMode, PythonDispatcher].

QuantizedCPU: registered at ../aten/src/ATen/native/quantized/cpu/qconv.cpp:1555 [kernel]
Quantiz

In [ ]:
model_static_quantized

QuantizedModel(
  (model_fp32): ResNet(
    (conv1): QuantizedConvReLU2d(3, 64, kernel_size=(7, 7), stride=(2, 2), scale=1.0, zero_point=0, padding=(3, 3))
    (bn1): Identity()
    (relu): Identity()
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): QuantizedConvReLU2d(64, 64, kernel_size=(3, 3), stride=(1, 1), scale=1.0, zero_point=0, padding=(1, 1))
        (bn1): Identity()
        (relu): Identity()
        (conv2): QuantizedConv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), scale=1.0, zero_point=0, padding=(1, 1))
        (bn2): Identity()
      )
      (1): BasicBlock(
        (conv1): QuantizedConvReLU2d(64, 64, kernel_size=(3, 3), stride=(1, 1), scale=1.0, zero_point=0, padding=(1, 1))
        (bn1): Identity()
        (relu): Identity()
        (conv2): QuantizedConv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), scale=1.0, zero_point=0, padding=(1, 1))
        (bn2): Identi

In [ ]:
model_static_quantized

In [ ]:
mmm = resnet18()
mmm.eval();

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
opt_model = torch.compile(mmm, backend="inductor")

In [ ]:
opt_model(x)

InternalTorchDynamoError: 

In [ ]:
bn = BN_Folder()

In [ ]:
aa = bn.fold(mmm)

In [ ]:
aa

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (bn1): Identity()
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): Identity()
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): Identity()
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): Identity()
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): Identity()
    )
  )
  (layer2): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (bn1): Identity()
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(128, 

In [ ]:
import torch
import torchvision.models as models
import torch.quantization

# Example with a standard model like ResNet
model = models.resnet18(pretrained=True)
#model.eval();

# Fuse Conv, BN, ReLU layers where applicable
# This step will vary based on your model architecture
#model = torch.quantization.fuse_modules(model, [['conv1', 'bn1', 'relu']], inplace=True)

# Set the quantization configuration for QAT
model.qconfig = torch.quantization.get_default_qat_qconfig('fbgemm')

# Prepare the model for QAT
torch.quantization.prepare_qat(model, inplace=True)

/home/HubensN/miniconda3/envs/prune/lib/python3.9/site-packages/torch/ao/quantization/observer.py:214: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(


ResNet(
  (conv1): Conv2d(
    3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
    (weight_fake_quant): FusedMovingAvgObsFakeQuantize(
      fake_quant_enabled=tensor([1]), observer_enabled=tensor([1]), scale=tensor([1.]), zero_point=tensor([0], dtype=torch.int32), dtype=torch.qint8, quant_min=-128, quant_max=127, qscheme=torch.per_channel_symmetric, reduce_range=False
      (activation_post_process): MovingAveragePerChannelMinMaxObserver(min_val=tensor([]), max_val=tensor([]))
    )
    (activation_post_process): FusedMovingAvgObsFakeQuantize(
      fake_quant_enabled=tensor([1]), observer_enabled=tensor([1]), scale=tensor([1.]), zero_point=tensor([0], dtype=torch.int32), dtype=torch.quint8, quant_min=0, quant_max=127, qscheme=torch.per_tensor_affine, reduce_range=True
      (activation_post_process): MovingAverageMinMaxObserver(min_val=inf, max_val=-inf)
    )
  )
  (bn1): BatchNorm2d(
    64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
    (ac

In [ ]:
from fastai.callback.core import Callback

class QuantizationAwareTrainingCallback(Callback):
    def after_epoch(self):
        print(self.model)
    def after_fit(self):
        # Convert the model to a fully quantized state for inference
        self.model.cpu()
        torch.quantization.convert(self.model, inplace=True)

In [ ]:
from fastai.learner import Learner
from fastai.vision.data import ImageDataLoaders
from torchvision.transforms import ToTensor

# Create a Learner
learner = Learner(dls, resnet18(), metrics=[accuracy])

# Train the model with QAT
learner.fit_one_cycle(5, 1e-2, cbs=[QuantizationAwareTrainingCallback()])

epoch,train_loss,valid_loss,accuracy,time
0,0.764580,19.327991,0.695535,00:07
1,0.636678,0.582403,0.694858,00:07
2,0.575387,0.535951,0.728687,00:07
3,0.525586,0.493728,0.754398,00:08
4,0.468614,0.460910,0.768606,00:07


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
x, y = dls.one_batch()

In [ ]:
learner.model(x.cpu())

NotImplementedError: Could not run 'quantized::conv2d.new' with arguments from the 'CPU' backend. This could be because the operator doesn't exist for this backend, or was omitted during the selective/custom build process (if using custom build). If you are a Facebook employee using PyTorch on mobile, please visit https://fburl.com/ptmfixes for possible resolutions. 'quantized::conv2d.new' is only available for these backends: [QuantizedCPU, QuantizedCUDA, BackendSelect, Python, FuncTorchDynamicLayerBackMode, Functionalize, Named, Conjugate, Negative, ZeroTensor, ADInplaceOrView, AutogradOther, AutogradCPU, AutogradCUDA, AutogradXLA, AutogradMPS, AutogradXPU, AutogradHPU, AutogradLazy, AutogradMeta, Tracer, AutocastCPU, AutocastCUDA, FuncTorchBatched, FuncTorchVmapMode, Batched, VmapMode, FuncTorchGradWrapper, PythonTLSSnapshot, FuncTorchDynamicLayerFrontMode, PythonDispatcher].

QuantizedCPU: registered at ../aten/src/ATen/native/quantized/cpu/qconv.cpp:1555 [kernel]
QuantizedCUDA: re

In [ ]:
model_quantized = torchvision.models.quantization.resnet18()

In [ ]:
model_quantized.eval();

In [ ]:
model_quantized.qconfig = torch.ao.quantization.get_default_qat_qconfig('x86')

In [ ]:
model_quantized.fuse_model()

In [ ]:
model_quantized = torch.ao.quantization.prepare_qat(model_quantized.train())

/home/HubensN/miniconda3/envs/prune/lib/python3.9/site-packages/torch/ao/quantization/observer.py:214: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(


In [ ]:
learn = Learner(dls, model_quantized, metrics=[accuracy])

In [ ]:
learn.fit(3)

epoch,train_loss,valid_loss,accuracy,time
0,1.064711,0.693148,0.695535,00:08
1,0.743257,0.693148,0.695535,00:08
2,0.700798,0.693148,0.695535,00:08


In [ ]:
learn.model.eval()
model_int8 = torch.ao.quantization.convert(learn.model)

RuntimeError: Unsupported qscheme: per_channel_affine

In [ ]:
learn.model.eval()
model_int8 = torch.ao.quantization.convert(learn.model)

# run the model, relevant calculations will happen in int8
res = model_int8(input_fp32)

In [ ]:
model_fp32_prepared.eval()
model_int8 = torch.ao.quantization.convert(model_fp32_prepared)

# run the model, relevant calculations will happen in int8
res = model_int8(input_fp32)

In [ ]:
import torch

# define a floating point model where some layers could benefit from QAT
class M(torch.nn.Module):
    def __init__(self):
        super().__init__()
        # QuantStub converts tensors from floating point to quantized
        self.quant = torch.ao.quantization.QuantStub()
        self.conv = torch.nn.Conv2d(1, 1, 1)
        self.bn = torch.nn.BatchNorm2d(1)
        self.relu = torch.nn.ReLU()
        # DeQuantStub converts tensors from quantized to floating point
        self.dequant = torch.ao.quantization.DeQuantStub()

    def forward(self, x):
        x = self.quant(x)
        x = self.conv(x)
        x = self.bn(x)
        x = self.relu(x)
        x = self.dequant(x)
        return x

# create a model instance
model_fp32 = M()

# model must be set to eval for fusion to work
model_fp32.eval()

# attach a global qconfig, which contains information about what kind
# of observers to attach. Use 'x86' for server inference and 'qnnpack'
# for mobile inference. Other quantization configurations such as selecting
# symmetric or asymmetric quantization and MinMax or L2Norm calibration techniques
# can be specified here.
# Note: the old 'fbgemm' is still available but 'x86' is the recommended default
# for server inference.
# model_fp32.qconfig = torch.ao.quantization.get_default_qconfig('fbgemm')
model_fp32.qconfig = torch.ao.quantization.get_default_qat_qconfig('x86')

# fuse the activations to preceding layers, where applicable
# this needs to be done manually depending on the model architecture
model_fp32_fused = torch.ao.quantization.fuse_modules(model_fp32,
    [['conv', 'bn', 'relu']])

# Prepare the model for QAT. This inserts observers and fake_quants in
# the model needs to be set to train for QAT logic to work
# the model that will observe weight and activation tensors during calibration.
model_fp32_prepared = torch.ao.quantization.prepare_qat(model_fp32_fused.train())

# run the training loop (not shown)
training_loop(model_fp32_prepared)

# Convert the observed model to a quantized model. This does several things:
# quantizes the weights, computes and stores the scale and bias value to be
# used with each activation tensor, fuses modules where appropriate,
# and replaces key operators with quantized implementations.
model_fp32_prepared.eval()
model_int8 = torch.ao.quantization.convert(model_fp32_prepared)

# run the model, relevant calculations will happen in int8
res = model_int8(input_fp32)